In [ ]:
# Import necessary packages
import woodwork as ww
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
import imblearn
from scipy.stats import f_oneway
import statsmodels.api as sm 
from statsmodels.formula.api import ols

In [ ]:
# Import train, validation and test set 
X =  pd.read_csv('../Processed datasets/After splitting/mrmr/500selected_four_categories.csv')
y  = pd.read_csv('../cleaned_imputed_split/y_train.csv')

X_val = pd.read_csv('../Processed datasets/After splitting/mrmr/500selected_val_four_categories.csv')
y_val = pd.read_csv('../cleaned_imputed_split/y_val.csv')

X_test = pd.read_csv('../Processed datasets/After splitting/mrmr/500selected_test_four_categories.csv')
y_test = pd.read_csv('../cleaned_imputed_split/y_test.csv')

feature_names = X.columns.tolist()

In [ ]:
# Replace 2 by 1 so 0 means non-diabetic and 1 means diabetic 
y.loc[y['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y.loc[y['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

y_val.loc[y_val['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y_val.loc[y_val['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

y_test.loc[y_test['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y_test.loc[y_test['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

#### Identifying columns that need rescaling

In [ ]:
# Load dictionary with datatypes 
import pickle 
with open('../cleaned_imputed_split/datatype_dictionary.pkl', 'rb') as f:
    datatypes = pickle.load(f)

In [ ]:
# Identify columns of datatype Double
double = {key: value for key, value in datatypes.items() if value == ww.logical_types.Double}

In [ ]:
# Identify columns of datatype Integer
integers = {key: value for key, value in datatypes.items() if value == ww.logical_types.Integer}

In [ ]:
# Add Doubles and Integers to a list
features_to_be_rescaled = list(double.keys()) + list(integers.keys())

In [ ]:
# Create a list of columns that need to be rescaled
dataframe_columns = X.columns.tolist()
rescaling = []

for i in features_to_be_rescaled:
    for j in dataframe_columns:
        if i in j:
            rescaling.append(j)

In [ ]:
# Rescale necessary columns
scaler = StandardScaler()
X[rescaling] = scaler.fit_transform(X[rescaling])
X_val[rescaling] = scaler.fit_transform(X_val[rescaling])
X_test[rescaling] = scaler.fit_transform(X_test[rescaling])

#### Evaluate cross-validation performance

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, make_scorer

# Convert the training and validation data to numpy arrays
X = X.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
y = y.to_numpy()

# Create an empty dictionary to save ROC scores
roc_scores = {}

# Create a list of the number of features to loop over 
n_features_list = [10, 50, 100, 200]

# Fit XGBoost classifier 
for n_features in n_features_list:
    # Wrap model with Recursive Feature Elimination
    rfe = RFE(estimator=XGBClassifier(), n_features_to_select=n_features)
    model = XGBClassifier()

    # Create pipeline with RFE and model
    pipeline = Pipeline(steps=[('s', rfe), ('m', model)])

    # Fit model to training data 
    pipeline.fit(X,y.ravel())
      
    # Evaluate model by performing cross-validation  
    scores = cross_val_score(pipeline, X,y.ravel(), cv=5, scoring='roc_auc')
    
    # Print average AUC score 
    print(scores)
    print("Average AUC:", np.mean(scores))

#### Evaluate performance of pre-optimized model on the validation set with a confusion matrix

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, make_scorer

# Convert the training and validation data to numpy arrays
X = X.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
y = y.to_numpy()

# Select the specified number of features using RFE
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=200)

# Create an XGBoost classifier model
model = XGBClassifier()

# Fit the pipeline model to the training data
pipeline = Pipeline(steps=[('s', rfe), ('m', model)])
pipeline.fit(X,y.ravel())

# Get the selected feature names from the RFE support mask
selected_f = [feature_names[i] for i, support in enumerate(rfe.support_) if support]
print(selected_f)

# Predict target for the validation data
y_pred_val = pipeline.predict(X_val)

# Predict probabilities for the positive class (class 1) in the validation data
y_prob_val = pipeline.predict_proba(X_val)[:,1]

# Calculate the ROC-AUC score for the validation data predictions
auc = roc_auc_score(y_val, y_prob_val)
print("AUC:",auc)


# Generate and plot the confusion matrix for the validation data predictions
cm = confusion_matrix(y_val, y_pred_val, labels=[0,1])
sns.heatmap(cm, annot=True, fmt='g', xticklabels=['No diabetes', 'Diabetes'], yticklabels=['No diabetes', 'Diabetes'])
plt.ylabel('Actual', fontsize=13)
plt.xlabel('Predicted', fontsize=13)
plt.title('Confusion Matrix', fontsize=17)
plt.show()

#### Hyperparameter tuning 

In [ ]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# Convert the training and validation data to numpy arrays
X = X.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
y = y.to_numpy()

# Select the specified number of features using RFE
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=200)

# Create an XGBoost classifier model
model = XGBClassifier()

# Fit the pipeline model to the training data
pipeline = Pipeline(steps=[('s', rfe), ('m', model)])

# Define a parameter grid for cross validation grid search 
param_grid = {'m__learning_rate':[0.01, 0.1, 0.3], # Learning rate for the XGBoost model
              'm__min_child_weight':[1,3,6], # Minimum child weight
              'm__subsample':[0.7,1], # Subsample ratio 
              's__n_features_to_select':[10,50, 100, 200]} # Number of feature to select using RFE 

# Perform grid search with cross-validation (cv=5) using ROC AUC as the scoring metric
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc', verbose=1)
grid_search.fit(X,y.ravel())

best_model = grid_search.best_estimator_

# Convert grid search results to a DataFrame
results = pd.DataFrame(grid_search.cv_results_)

# Create a pivot table to reorganize the grid search results for visualization
pivot_table = results.pivot_table(
    values='mean_test_score',
    index=['param_m__subsample', 'param_m__min_child_weight'],
    columns=['param_m__learning_rate', 'param_s__n_features_to_select']
)

# Plot the grid search results in a clustermap
sns.clustermap(pivot_table, annot=True, cmap='viridis')

In [ ]:
clustergrid = sns.clustermap(pivot_table, annot=True, cmap='rocket_r')
clustergrid.fig.savefig('hyperparameter_tuning_XGBoost.png')

#### Evaluate StratifiedKFoldCross results

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import roc_auc_score, confusion_matrix

# Convert the training and validation data to numpy arrays
X = X.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
y = y.to_numpy()

# Select the specified number of features using RFE
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=100)

# Create an XGBoost classifier model
model = XGBClassifier(learning_rate=0.1)

# Fit the pipeline model to the training data
pipeline = Pipeline(steps=[('s', rfe), ('m', model)])

# Split the data into 5 folds 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []

for train_index, test_index in skf.split(X,y):
    # Split the data into training and testing sets for the current fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the pipeline model on the training data
    pipeline.fit(X_train, y_train.ravel())

    # Predict probabilities for the positive class (class 1) on the test data
    probas_ = pipeline.predict_proba(X_test)[:,1]

    # Calculate the ROC-AUC score for the current fold
    auc_score = roc_auc_score(y_test, probas_)

    # Append the AUC score to the list
    auc_scores.append(auc_score)

# Print average AUC score across 5 folds 
print(auc_scores)
average_auc = np.mean(auc_scores)
print(f'Average ROC-AUC Score: {average_auc}')

#### Evaluate performance on test set

In [ ]:
# Import train, validation and test set 
X =  pd.read_csv('../Processed datasets/After splitting/mrmr/300selected_four_categories.csv')
y  = pd.read_csv('../cleaned_imputed_split/y_train.csv')

X_val = pd.read_csv('../Processed datasets/After splitting/mrmr/300selected_val_four_categories.csv')
y_val = pd.read_csv('../cleaned_imputed_split/y_val.csv')

X_test = pd.read_csv('../Processed datasets/After splitting/mrmr/500selected_test_four_categories.csv')
y_test = pd.read_csv('../cleaned_imputed_split/y_test.csv')

feature_names = X.columns.tolist()

In [ ]:
# Replace 2 by 1 so 0 means non-diabetic and 1 means diabetic 
y.loc[y['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y.loc[y['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

y_val.loc[y_val['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y_val.loc[y_val['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

y_test.loc[y_test['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y_test.loc[y_test['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

In [ ]:
X[rescaling] = scaler.fit_transform(X[rescaling])
X_val[rescaling] = scaler.fit_transform(X_val[rescaling])
X_test[rescaling] = scaler.fit_transform(X_test[rescaling])

In [ ]:
X_original = X_test.copy()

In [ ]:
# Merge train and validation set 
X_train_full = pd.concat([X, X_val], ignore_index=True, axis=0)
y_train_full = pd.concat([y, y_val], ignore_index=True, axis=0)

# Convert the training and validation data to numpy arrays
X = X_train_full.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
y = y_train_full.to_numpy()

# Select the specified number of features using RFE
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=100)

# Create an XGBoost classifier model
model = XGBClassifier(learning_rate=0.1)

# Fit the pipeline model to the training data
pipeline = Pipeline(steps=[('s', rfe), ('m', model)])
pipeline.fit(X,y.ravel())

# Get the selected feature names from the RFE support mask
selected_f = [feature_names[i] for i, support in enumerate(rfe.support_) if support]
print(selected_f)

# Predict target for the test data
y_pred_test = pipeline.predict(X_test)

# Predict probabilities for the positive class (class 1) in the test data
y_prob_test = pipeline.predict_proba(X_test)[:,1]

# Calculate the ROC-AUC score for the test data predictions
auc = roc_auc_score(y_test, y_prob_test)
print("AUC:",auc)

# Generate and plot the confusion matrix for the test data predictions
cm = confusion_matrix(y_test, y_pred_test, labels=[0,1])
sns.heatmap(cm, annot=True, fmt='g', xticklabels=['No diabetes', 'Diabetes'], yticklabels=['No diabetes', 'Diabetes'], cmap='Blues')
plt.ylabel('Actual', fontsize=13)
plt.xlabel('Predicted', fontsize=13)
plt.title('Confusion Matrix', fontsize=17)
plt.show()

In [ ]:
import shap

In [ ]:
X_test_transformed = pipeline.named_steps['s'].transform(X_test)

explainer = shap.Explainer(pipeline.named_steps['m'], X_test_transformed)

# Calculate SHAP values
shap_values = explainer(X_test_transformed)

# This plot shows the importance of each feature and its impact on the model's predictions
shap.summary_plot(shap_values, X_test_transformed)

## Comparison with features Engineered by TNO

In [ ]:
import pickle 
# Save the 100 selected features by the wrapped XGBoost model
with open('../optimized_model_results/100_selected_features_XGBoost.pkl', 'wb') as f:
    pickle.dump(selected_f, f)

In [ ]:
# Load Whitehall dictionary from files to use
with open('../Processed datasets/feature_dictionary.pkl', 'rb') as f:
    columns_whitehall = pickle.load(f)

In [ ]:
# Load 100 selected features from files to use
with open('../optimized_model_results/100_selected_features_XGBoost.pkl', 'rb') as f:
    XGBoost_selected_features = pickle.load(f)

In [ ]:
# Import the features that were inluded by TNO 
selected_variables_TNO = pd.ExcelFile('../TNO_datasets/Selected Variables.xlsx')  
predictors = pd.read_excel(selected_variables_TNO, 'Predictors')
predictors_phase_9 = predictors.dropna(subset=['Phase 9'])
feature_names_TNO = predictors_phase_9['Phase 9'].tolist()

In [ ]:
# Initialize an empty list to store the matching features
found_features = []

# Iterate over each feature name in the feature_names_TNO list
for feature in feature_names_TNO:
    # Check if the feature is part of any of the XGBoost selected features
    if any(feature in combination for combination in XGBoost_selected_features):
        # If found, append the feature to the found_features list
        found_features.append(feature)

count = len(found_features)

print(found_features)

In [ ]:
# List of transformation primitives to search for in the feature names
trans_primitives = ['SQUARE_ROOT', 'PERCENTILE', 'AND(', 'OR(', 'NOT', '*', '-', '+']

# Initialize a dictionary to store counts of features containing each primitive
counts = {primitive: [] for primitive in trans_primitives}

# Iterate over each transformation primitive
for primitive in trans_primitives:
    # Iterate over each selected feature name
    for feature in selected_f:
         # If the primitive is found in the feature name, add the feature to the counts dictionary
        if primitive in feature:
            counts[primitive].append(feature)

# Print the number of features containing each transformation primitive
for primitive, count in counts.items():
    print(f"Number of strings containing '{primitive}': {len(count)}")

print(f"The number of raw features is:", (len(selected_f) - sum(len(count) for count in counts.values())))

In [ ]:
import re 
from collections import defaultdict, Counter

# Extract the individual column names from a combined feature 
def extract_columns(feature):
    feature = feature.replace("AND(", "").replace(")", "")
    return re.split(r'\s*[\+\-\*,]\s', feature)

# Map each column name to its corresponding category 
column_to_category = {}
for category, columns in columns_whitehall.items():
    for column in columns:
        column_to_category[column] = category

# Count the number of times that combinations occur 
category_combinations = Counter()

for feature in selected_f:
    columns = extract_columns(feature)
    involved_categories = {column_to_category.get(column) for column in columns if column and column in column_to_category}
    involved_categories.discard(None)

    category_combinations[tuple(sorted(involved_categories))] +=1
    
# Print the combinations of categories and their counts 
for combination, count in category_combinations.items():
    print(f"Combination {combination}: {count} feature(s)")

In [ ]:
with open('../optimized_model_results/50_selected_features_LR.pkl', 'rb') as f:
    selected_features_LR = pickle.load(f)

In [ ]:
XGBoost_set = set(selected_features_XGBoost)
LR_set = set(selected_features_LR)

In [ ]:
# Determine how many selected features XGBoost and LR have in common
common_features = XGBoost_set.intersection(LR_set)
common_feature_count = len(common_features)
common_feature_count

In [ ]:
common_features

#### In depth error analysis TP, FN, and FP cases

In [ ]:
# Import train, validation and test set 
X =  pd.read_csv('../Processed datasets/After splitting/mrmr/500selected_four_categories.csv')
y  = pd.read_csv('../cleaned_imputed_split/y_train.csv')

X_val = pd.read_csv('../Processed datasets/After splitting/mrmr/500selected_val_four_categories.csv')
y_val = pd.read_csv('../cleaned_imputed_split/y_val.csv')

X_test = pd.read_csv('../Processed datasets/After splitting/mrmr/500selected_test_four_categories.csv')
y_test = pd.read_csv('../cleaned_imputed_split/y_test.csv')


In [ ]:
# Replace 2 by 1 so 0 means non-diabetic and 1 means diabetic 
y.loc[y['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y.loc[y['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

y_val.loc[y_val['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y_val.loc[y_val['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

y_test.loc[y_test['diabetic_outcome'] == 0, 'diabetic_outcome'] = 0
y_test.loc[y_test['diabetic_outcome'] == 2, 'diabetic_outcome'] = 1

In [ ]:
X[rescaling] = scaler.fit_transform(X[rescaling])
X_val[rescaling] = scaler.fit_transform(X_val[rescaling])
X_test[rescaling] = scaler.fit_transform(X_test[rescaling])

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, make_scorer
# Merge train and validation set 
X_train_full = pd.concat([X, X_val], ignore_index=True, axis=0)
y_train_full = pd.concat([y, y_val], ignore_index=True, axis=0)

# Convert the training and validation data to numpy arrays
X = X.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
y = y.to_numpy()

# Select the specified number of features using RFE
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=100)

# Create an XGBoost classifier model
model = XGBClassifier(learning_rate=0.1)

# Fit the pipeline model to the training data
pipeline = Pipeline(steps=[('s', rfe), ('m', model)])
pipeline.fit(X,y.ravel())

# Get the selected feature names from the RFE support mask
selected_f = [feature_names[i] for i, support in enumerate(rfe.support_) if support]
print(selected_f)

# Predict target for the test data
y_pred_test = pipeline.predict(X_test)

# Predict probabilities for the positive class (class 1) in the test data
y_prob_test = pipeline.predict_proba(X_test)[:,1]

# Calculate the ROC-AUC score for the test data predictions
auc = roc_auc_score(y_test, y_prob_test)
print("AUC:",auc)

# Generate and plot the confusion matrix for the test data predictions
cm = confusion_matrix(y_test, y_pred_test, labels=[0,1])
sns.heatmap(cm, annot=True, fmt='g', xticklabels=['No diabetes', 'Diabetes'], yticklabels=['No diabetes', 'Diabetes'], cmap='Blues')
plt.ylabel('Actual', fontsize=13)
plt.xlabel('Predicted', fontsize=13)
plt.title('Confusion Matrix', fontsize=17)
plt.show()

In [ ]:
# Create a DataFrame from the test feature set (X_test) with columns named after the features
X_test_df = pd.DataFrame(X_test, columns=feature_names)

# Select only the features that were selected by RFE
X_test_df = X_test_df[selected_f]

# Add a column to the DataFrame with the predicted labels for the test data
X_test_df['predicted'] = y_pred_test

# Add a column to the DataFrame with the actual labels for the test data
X_test_df['actual'] = y_test

In [ ]:
# Define conditions for different prediction outcomes
# (True Positive, False Positive, False Negative, True Negative)
conditions = [(X_test_df['predicted'] == 1) & (X_test_df['actual'] == 1),
              (X_test_df['predicted'] == 1) & (X_test_df['actual'] == 0),
              (X_test_df['predicted'] == 0) & (X_test_df['actual'] == 1),
              (X_test_df['predicted'] == 0) & (X_test_df['actual'] == 0)]

# Define labels corresponding to each condition
# 1: True Positive (TP)
# 2: False Positive (FP)
# 3: False Negative (FN)
# 4: True Negative (TN)

labels = [1,2,3,4]

In [ ]:
# Create a new column 'confusion_matrix_category' in the test DataFrame
# This column will categorize each prediction based on the conditions defined earlier
# np.select applies the conditions in the given order and assigns the corresponding label
# If none of the conditions are met, the default value (np.nan) is assigned
X_test_df['confusion_matrix_category'] = np.select(conditions, labels, default=np.nan)

In [ ]:
# Drop True Negatives
X_test_df = X_test_df[X_test_df['confusion_matrix_category'] != 4]

# Drop created columns with the predicted and actual labels 
X_test_df.drop(columns=['predicted', 'actual'], inplace=True)

In [ ]:
# Dictionary to store the ANOVA results
results = {}

# Iterate over each feature column in the test DataFrame (excluding the last column 'confusion_matrix_category')
# Determine if there are statistically significant differences in feature values between the groups defined in the column 'confusion_matrix_category'
for feature in X_test_df.columns[:-1]:
    model = ols(f'Q("{feature}") ~ C(confusion_matrix_category)', data = X_test_df).fit()

    # Perform ANOVA (Analysis of Variance) on the OLS model
    anova_result = sm.stats.anova_lm(model, typ=2)
    p_value = anova_result.at['C(confusion_matrix_category)', 'PR(>F)']
    results[feature] = p_value

# Set p < 0.05 to print statistically different features 
for feature, p_val in results.items():
    if p_val < 0.05:
        print(f"{feature}: p-value = {p_val}")


In [ ]:
# Create boxplot of top predictor 
sns.boxplot(x='confusion_matrix_category', y='JCHOLMED * JP10CVD', data=X_test_df)
plt.title('JCHOLDMED * JP10CVD boxplot per category')
plt.xlabel('Category')
group_labels = ['True positive', 'False positive', 'False negative']
plt.xticks(ticks=plt.gca().get_xticks(), labels=group_labels)
plt.show()

In [ ]:
# Create boxplot of top predictor 
sns.boxplot(x='confusion_matrix_category', y='JGLUC_2H * JP10CVD', data=X_test_df)
plt.title('JGLUC_2H * JP10CVD boxplot per category')
plt.xlabel('Category')
group_labels = ['True positive', 'False positive', 'False negative']
plt.xticks(ticks=plt.gca().get_xticks(), labels=group_labels)
plt.show()

In [ ]:
# Create boxplot of top predictor 
sns.boxplot(x='confusion_matrix_category', y='JFAT * JP10CVD', data=X_test_df)
plt.title('JFAT * JP10CVD boxplot per category')
plt.xlabel('Category')
group_labels = ['True positive', 'False positive', 'False negative']
plt.xticks(ticks=plt.gca().get_xticks(), labels=group_labels)
plt.show()